In [17]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build

Loads API key from .env

In [18]:
load_dotenv()
api_key = os.getenv("YOUTUBE_API_KEY")
youtube = build('youtube', 'v3', developerKey=api_key)

In [19]:
def get_full_data(video_url):
    video_id = video_url.split("v=")[1]
    video_request = youtube.videos().list(part="snippet,statistics", id=video_id)
    video_response = video_request.execute()
    
    if not video_response['items']:
        return None
        
    video_item = video_response['items'][0]

    artist_name = video_item['snippet']['channelTitle'].replace(" - Topic", "")
    search_request = youtube.search().list(
        q=f"{artist_name} official", 
        type="channel", 
        part="id", 
        maxResults=1
    )
    search_response = search_request.execute()
    
    if not search_response['items']:
        official_channel_id = video_item['snippet']['channelId']
    else:
        official_channel_id = search_response['items'][0]['id']['channelId']    
    
    channel_request = youtube.channels().list(part="snippet,statistics", id=official_channel_id)
    channel_response = channel_request.execute()
    channel_item = channel_response['items'][0]

    return {
        "Video ID": video_id,
        "Title": video_item['snippet']['title'],
        "Channel Name": channel_item['snippet']['title'],
        "Upload Date": video_item['snippet']['publishedAt'],
        "View Count": video_item['statistics'].get('viewCount', 0),
        "Likes": video_item['statistics'].get('likeCount', 0),
        "Comments": video_item['statistics'].get('commentCount', 0),
        "Tags": video_item['snippet'].get('tags', []),
        "Category ID": video_item['snippet']['categoryId'],
        "Channel Subscriber Count": channel_item['statistics']['subscriberCount'],
        "Channel Total View Count": channel_item['statistics']['viewCount'],
        "Channel Video Count": channel_item['statistics']['videoCount'],
        "Channel Age (Created At)": channel_item['snippet']['publishedAt']
    }

In [20]:
video_url = "https://www.youtube.com/watch?v=Bf01riuiJWA"
video_data = get_full_data(video_url)

for k, v in video_data.items():
    print(f"{k}: {v}")

Video ID: Bf01riuiJWA
Title: Exit Music (For A Film)
Channel Name: Radiohead
Upload Date: 2016-04-06T21:57:35Z
View Count: 40237876
Likes: 551777
Comments: 8525
Tags: ['Radiohead', 'OK Computer', 'Exit Music (For A Film)']
Category ID: 10
Channel Subscriber Count: 4940000
Channel Total View Count: 4335644239
Channel Video Count: 176
Channel Age (Created At): 2005-10-08T15:10:46Z
